In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import os
import openai


# simple prompt

In [3]:
from openai import OpenAI
client = OpenAI()

# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
#   ]
# )

# print(completion.choices[0].message)

# Prompt + LLM

In [16]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
model = ChatOpenAI()
chain = prompt | model
chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

# Agent

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [22]:
from langchain.agents import tool
from datetime import date

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool
def get_todays_date() -> str:
    """Returns todays date, use this for any questions related to todays date."""
    return str(date.today())

get_word_length.invoke("abc")

3

In [23]:
tools = [get_word_length, get_todays_date]

In [24]:
from langchain.agents import AgentType, initialize_agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [31]:
# set up simple chain
prompt = ChatPromptTemplate.from_template("{question}")
model = ChatOpenAI()
chain = prompt | model

### Test: example 1

In [28]:

chain.invoke({"question": "what is today's date?"})

AIMessage(content="Today's date is September 30, 2021.")

In [25]:
agent("what is today's date?")



> Entering new AgentExecutor chain...
I can use the `get_todays_date` tool to get today's date for you.
Action:
```{"action": "get_todays_date", "action_input": {}}```
Observation: 2024-01-25
Thought:Today's date is January 25, 2024.

> Finished chain.


{'input': "what is today's date?",
 'output': "Today's date is January 25, 2024."}

### Test: example 2

In [30]:
chain.invoke({"question": "How many letters are there in pneumonoultramicroscopicsilicovolcanoconiosis?"})

AIMessage(content='There are 45 letters in the word "pneumonoultramicroscopicsilicovolcanoconiosis".')

In [29]:
agent("How many letters are there in pneumonoultramicroscopicsilicovolcanoconiosis?")



> Entering new AgentExecutor chain...
This word is known to be one of the longest words in the English language. To find out how many letters are in "pneumonoultramicroscopicsilicovolcanoconiosis", I can use the `get_word_length` tool. Let me do that for you.
Action:
```json
{
  "action": "get_word_length",
  "action_input": {
    "word": "pneumonoultramicroscopicsilicovolcanoconiosis"
  }
}
```
Observation: 45
Thought:The word "pneumonoultramicroscopicsilicovolcanoconiosis" has 45 letters.
Action:
```json
{
  "action": "Final Answer",
  "action_input": "There are 45 letters in pneumonoultramicroscopicsilicovolcanoconiosis."
}
```

> Finished chain.


{'input': 'How many letters are there in pneumonoultramicroscopicsilicovolcanoconiosis?',
 'output': 'There are 45 letters in pneumonoultramicroscopicsilicovolcanoconiosis.'}

# Agent with searching capability

In [5]:
from langchain.agents import load_tools
tools = load_tools(["ddg-search", "llm-math", "wikipedia"], llm=llm)

In [7]:
from langchain.agents import AgentType, initialize_agent
agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)

/Users/shaluo/Documents/GitHub/agent/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [8]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

duckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.
Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [duckduckgo_search, Calculator, Wikipedia]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Tho

In [44]:
query = """
Who is the current CEO of Tesla? When was he born?
"""

agent.run(query)

/Users/shaluo/Documents/GitHub/agent/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I need to find information about the current CEO of Tesla and his birthdate.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$232 billion as of December 2023, according to the Bloomberg Billionaires Index, and $254 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his

'The current CEO of Tesla is Elon Musk. He was born on June 28, 1971.'

# Query a SQL DB

In [46]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [47]:
from langchain_community.utilities import SQLDatabase

In [48]:
db = SQLDatabase.from_uri("sqlite:///./Chinook.db")

In [49]:
def get_schema(_):
    return db.get_table_info()

In [50]:
def run_query(query):
    return db.run(query)

In [58]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [59]:
sql_response.invoke({"question": "How many employees are there?"})

'SELECT COUNT(*) FROM employees;'

In [63]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [64]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | llm
)

In [65]:
full_chain.invoke({"question": "How many employees are there?"})

OperationalError: (sqlite3.OperationalError) no such table: employees
[SQL: SELECT COUNT(*) AS num_employees
FROM employees]
(Background on this error at: https://sqlalche.me/e/20/e3q8)